## Training SageMaker Models using the DGL with MXNet backend
The **SageMaker Python SDK** makes it easy to train DGL models. In this example, we train a simple graph neural network using the [DMLC DGL API](https://github.com/dmlc/dgl.git) and the [cora dataset](https://relational.fit.cvut.cz/dataset/CORA). The cora dataset describes a citation network. The cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. The task at hand is to train a node classification model using Cora dataset. 

For more details about Graph Neural Network and this example please refer to https://docs.dgl.ai/en/latest/tutorials/models/1_gnn/1_gcn.html

### Prepare
First we need to install necessary packages.

In [ ]:
!conda install -y boto3
!conda install -c anaconda -y botocore

### Setup
We need to define a few variables that will be needed later in the example.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sess.default_bucket()

# Location to put your custom code.
custom_code_upload_location = 'customcode'

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### The training script
The mxnet_gcn.py script provides all the code we need for training a SageMaker model. 

In [ ]:
!cat mxnet_gcn.py

### SageMaker's  estimator class
The SageMaker Estimator allows us to run single machine in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role. We also provide a few other parameters. train_instance_count and train_instance_type determine the number and type of SageMaker instances that will be used for the training job. The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the mxnet_gcn.py script above.

Here we can use the official docker image for this example, please see https://github.com/aws/sagemaker-mxnet-container for more information.


In [ ]:
from sagemaker.mxnet.estimator import MXNet

CODE_PATH = 'mxnet_gcn.py'

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
docker_name = 'beta-mxnet-training'
docker_tag = '1.6.0-py3-gpu-build'
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, docker_name, docker_tag)
print(image)

params = {}
params['dataset'] = 'cora'
estimator = MXNet(entry_point=CODE_PATH,
                        role=role, 
                        train_instance_count=1, 
                        train_instance_type='ml.p3.2xlarge',
                        image_name=image,
                        hyperparameters=params,
                        sagemaker_session=sess)

### Running the Training Job
After we've constructed our Estimator object, we can fit it using sagemaker (The dataset will be automatically downloaded). Below we run SageMaker training on one channels: training-code, the code to run.

In [ ]:
estimator.fit()

## Output
You can get the model training output from the Sagemaker Console by searching for the training task and looking for the address of 'S3 model artifact'